In [5]:
import common_utils
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [10]:
# LU raster files
raster_file = '../lu_rasters/ESRI_LULC_2023_Rome.tif'

# Rome centroids
rome_points = "../lst_centroids/msg_lst_rome_5k_centroids.gpkg"

# Buffer radius in degrees (approximately 5000 meters)
buffer_radius = 0.05

In [11]:
# Define LULC remap dictionary
lulc_remap = {
    'lulc_0': 'no_data',
    'lulc_1': 'water',
    'lulc_2': 'trees',
    'lulc_4': 'flooded_veg',
    'lulc_5': 'crop',
    'lulc_7': 'built_area',
    'lulc_8': 'bare_ground',
    'lulc_9': 'snow_or_ice',
    'lulc_10': 'clouds',
    'lulc_11': 'range_land'
}


In [ ]:
gdf = gpd.read_file(rome_points)  # Read the .gpkg file

gdf.shape

In [13]:
gdf['lulc_proportions'] = common_utils.calculate_lulc_proportions(raster_file, gdf['geometry'], buffer_radius, lulc_remap)

In [14]:
# Expand LULC proportions into separate columns
lulc_df = pd.DataFrame(gdf['lulc_proportions'].tolist())
final_df = gpd.GeoDataFrame(pd.concat([gdf.drop(columns=['lulc_proportions']), lulc_df], axis=1),
                            geometry='geometry', crs="EPSG:4326")

In [15]:
final_df.shape

(80, 11)

In [16]:
# Summary statistics for all columns in the final GeoDataFrame
summary_stats = final_df.describe()

# Print the summary statistics
print(summary_stats)

         no_data      water      trees   flooded_veg       crop  built_area  \
count  80.000000  80.000000  80.000000  8.000000e+01  80.000000   80.000000   
mean    0.197339   0.030297   0.085282  8.086019e-07   0.279647    0.341370   
std     0.026633   0.100295   0.095198  2.384011e-06   0.175721    0.167830   
min     0.125772   0.000000   0.000248  0.000000e+00   0.005039    0.050070   
25%     0.177716   0.000288   0.020590  0.000000e+00   0.130125    0.208337   
50%     0.216991   0.001370   0.058859  0.000000e+00   0.278443    0.332168   
75%     0.217014   0.013635   0.104280  0.000000e+00   0.404295    0.462748   
max     0.218392   0.708143   0.493681  9.392620e-06   0.613449    0.708325   

       bare_ground  snow_or_ice  clouds  range_land  
count    80.000000         80.0    80.0   80.000000  
mean      0.000687          0.0     0.0    0.065378  
std       0.001936          0.0     0.0    0.032442  
min       0.000000          0.0     0.0    0.010159  
25%       0.000000

In [18]:
keep_columns = ['geometry', 'no_data', 'water', 'trees', 'flooded_veg', 'crop', 'built_area', 'bare_ground', 'snow_or_ice', 'clouds', 'range_land']
final_df = final_df[keep_columns]

final_df.set_crs(epsg=4326, inplace=True)

output_filepath = '../lu_profiles/rome_2023_landuse_profile_5000m.gpkg'

# Save to file or use further
final_df.to_file(output_filepath, driver='GPKG')

final_df.shape

(80, 11)

In [19]:
final_df.head()

,geometry,no_data,water,trees,flooded_veg,crop,built_area,bare_ground,snow_or_ice,clouds,range_land
0,POINT (12.25 42.05),0.132022,0.022978,0.044921,0.0,0.595550,0.189452,0.000000,0.0,0.0,0.015077
1,POINT (12.3 42.05),0.177677,0.000143,0.036355,0.0,0.585212,0.182375,0.000000,0.0,0.0,0.018237
2,POINT (12.35 42.05),0.178449,0.000134,0.055930,0.0,0.393311,0.332778,0.000000,0.0,0.0,0.039399
3,POINT (12.4 42.05),0.177711,0.000164,0.100341,0.0,0.367734,0.314093,0.000000,0.0,0.0,0.039957
4,POINT (12.45 42.05),0.177717,0.000030,0.122215,0.0,0.443907,0.223900,0.000002,0.0,0.0,0.032229
